<a href="https://colab.research.google.com/github/MarioAvolio/FoodX-251-Classification-ConvNext-vs-SwinTransformer/blob/main/FoodX_251_Similarity_Retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q torch_snippets
from torch_snippets import *
from torchvision.datasets import MNIST
from torchvision import transforms
from google.colab import drive
import pandas as pd
import numpy as np
import cv2
device = 'cuda' if torch.cuda.is_available() else 'cpu'

!pip install torch_summary
from torchsummary import summary

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.2/58.2 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.8/202.8 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 67.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 58.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.8/98.8 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.1/174.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.5/468.5 kB 40.1 MB/s eta 0:00:

In [2]:


NUM_CLASSES=251

G_DRIVE='/content/gdrive/'
PROJ_PATH= G_DRIVE+"MyDrive/Visual-Proj/"

# ----------------- tar
TRAIN_PATH = PROJ_PATH + "train.tar"
VAL_PATH = PROJ_PATH + "val.tar"
VAL_DEGRADED_PATH = PROJ_PATH + "val_degraded.zip"
NOISE_TRAINING_DATA_PATH = PROJ_PATH + "noise.zip"
ANNOTATION_PATH = PROJ_PATH + "annot.tar"


# ---------------- local data extracted
TRAIN_PATH_LOCAL="/content/train_set/"
VAL_DEGRADED_PATH_LOCAL="/content/val_set_degraded/"
VAL_PATH_LOCAL="/content/val_set/"
NOISED_PATH_LOCAL="/content/noise/"


# --------------- csv file
ANNOTATION_PATH_CLASS_LOCAL="/content/class_list.txt"
ANNOTATION_PATH_VALIDATION="/content/val_info.csv"
ANNOTATION_PATH_TRAIN_CLEANED=PROJ_PATH+"train_info_cleaned.csv"
ANNOTATION_PATH_BALANCED_TRAIN = PROJ_PATH + "balanced_train_info.csv"






In [3]:
def get_img_from_path(path):
  return cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)  

def get_random_sample_from_class(data, class_type):
  return data[data.TYPE==class_type].sample(1)

In [4]:
def initilize_path_into_val_dataset(row):
  return VAL_PATH_LOCAL + row.NAME

def initilize_path_into_val_dataset_degraded(row):
  return VAL_DEGRADED_PATH_LOCAL + row.NAME

In [5]:
def initilize_path_into_dataset(row):
  if row.NOISED == 0:
    row.PATH = TRAIN_PATH_LOCAL + row.NAME
  elif row.NOISED == 1:
    row.PATH = NOISED_PATH_LOCAL + row.NAME

  return row.PATH

In [6]:
def one_hot(data):
  y = pd.get_dummies(data.TYPE, prefix='Type')
  data = data.drop('TYPE', axis=1)
  data = pd.concat([data, y], axis=1) # add new hot-encoded columns 
  return data

In [7]:
def bound_instances_by_class(class_type,data):
  g = data[data.TYPE == class_type]
  g = g.sample(frac=0.2)
  data.drop(g.index, axis=0, inplace=True)
  
  return g.reset_index(drop=True)

In [121]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, classification_report

def plot_confusion_matrix(y_preds, y_true, labels=None):
  cm = confusion_matrix(y_true, y_preds, normalize="true")
  fig, ax = plt.subplots(figsize=(6, 6))
  disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[x for x in range(NUM_CLASSES)])
  disp.plot(cmfrom sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, classification_report



In [8]:
drive.mount(G_DRIVE, force_remount=True)

Mounted at /content/gdrive/


In [9]:
!tar -xf $TRAIN_PATH --skip-old-files 
!tar -xf $VAL_PATH --skip-old-files 
!tar -xf $ANNOTATION_PATH --skip-old-files 
!unzip -nqq $VAL_DEGRADED_PATH  
!unzip -nqq $NOISE_TRAINING_DATA_PATH  

# DATA

In [10]:
colnames=['ID', 'TYPE'] 
class_list = pd.read_csv(ANNOTATION_PATH_CLASS_LOCAL, delim_whitespace=True, index_col=False, header=None, names=colnames).set_index('ID')
id_to_type = class_list["TYPE"].to_dict()
type_to_id = {v: k for k, v in id_to_type.items()}

In [11]:
train_info = pd.read_csv(ANNOTATION_PATH_BALANCED_TRAIN)#.sample(frac=0.05).reset_index(drop=True) # shuffle
name_to_type_train=train_info.set_index("NAME").to_dict()["TYPE"]
type_to_name_train = {v: k for k, v in name_to_type_train.items()}

In [12]:
train_info

,NAME,TYPE,NOISED
0,train_050929.jpg,149,1
1,train_068867.jpg,205,0
2,train_079959.jpg,61,0
3,train_058377.jpg,85,0
4,train_002190.jpg,140,1
...,...,...,...
128256,train_083140.jpg,139,0
128257,train_101019.jpg,174,0
128258,train_001371.jpg,58,1
128259,train_062550.jpg,2,0


In [13]:
colnames=['NAME', 'TYPE'] 
test_info = pd.read_csv(ANNOTATION_PATH_VALIDATION, index_col=False, header=None, names=colnames)
name_to_type_val=test_info.set_index("NAME").to_dict()["TYPE"]
type_to_name_val = {v: k for k, v in name_to_type_val.items()}
# type_to_name_val

In [14]:
test_info = test_info.sample(frac=1).reset_index(drop=True) # shuffle

In [15]:
test_info

,NAME,TYPE
0,val_006911.jpg,101
1,val_006353.jpg,10
2,val_004340.jpg,180
3,val_008337.jpg,76
4,val_011061.jpg,216
...,...,...
11989,val_005375.jpg,190
11990,val_005389.jpg,161
11991,val_007327.jpg,98
11992,val_010901.jpg,217


In [16]:
test_info["PATH"] = 0
test_info_degraded = test_info.copy()
test_info_degraded["PATH"] = test_info.apply(initilize_path_into_val_dataset_degraded, axis=1)
test_info["PATH"] = test_info.apply(initilize_path_into_val_dataset, axis=1)


In [17]:
test_info

,NAME,TYPE,PATH
0,val_006911.jpg,101,/content/val_set/val_006911.jpg
1,val_006353.jpg,10,/content/val_set/val_006353.jpg
2,val_004340.jpg,180,/content/val_set/val_004340.jpg
3,val_008337.jpg,76,/content/val_set/val_008337.jpg
4,val_011061.jpg,216,/content/val_set/val_011061.jpg
...,...,...,...
11989,val_005375.jpg,190,/content/val_set/val_005375.jpg
11990,val_005389.jpg,161,/content/val_set/val_005389.jpg
11991,val_007327.jpg,98,/content/val_set/val_007327.jpg
11992,val_010901.jpg,217,/content/val_set/val_010901.jpg


In [18]:
test_info_degraded

,NAME,TYPE,PATH
0,val_006911.jpg,101,/content/val_set_degraded/val_006911.jpg
1,val_006353.jpg,10,/content/val_set_degraded/val_006353.jpg
2,val_004340.jpg,180,/content/val_set_degraded/val_004340.jpg
3,val_008337.jpg,76,/content/val_set_degraded/val_008337.jpg
4,val_011061.jpg,216,/content/val_set_degraded/val_011061.jpg
...,...,...,...
11989,val_005375.jpg,190,/content/val_set_degraded/val_005375.jpg
11990,val_005389.jpg,161,/content/val_set_degraded/val_005389.jpg
11991,val_007327.jpg,98,/content/val_set_degraded/val_007327.jpg
11992,val_010901.jpg,217,/content/val_set_degraded/val_010901.jpg


In [19]:
train_info["PATH"] = 0
train_info["PATH"] = train_info.apply(initilize_path_into_dataset, axis=1)

In [20]:
train_info

,NAME,TYPE,NOISED,PATH
0,train_050929.jpg,149,1,/content/noise/train_050929.jpg
1,train_068867.jpg,205,0,/content/train_set/train_068867.jpg
2,train_079959.jpg,61,0,/content/train_set/train_079959.jpg
3,train_058377.jpg,85,0,/content/train_set/train_058377.jpg
4,train_002190.jpg,140,1,/content/noise/train_002190.jpg
...,...,...,...,...
128256,train_083140.jpg,139,0,/content/train_set/train_083140.jpg
128257,train_101019.jpg,174,0,/content/train_set/train_101019.jpg
128258,train_001371.jpg,58,1,/content/noise/train_001371.jpg
128259,train_062550.jpg,2,0,/content/train_set/train_062550.jpg


In [21]:
from sklearn.model_selection import train_test_split

# train, val = train_test_split(train_info.NAME.unique(), test_size=0.2, random_state=42, shuffle=True) # an img can live or in val or in train but not in both!

In [22]:
# train, val

In [23]:
# list(set(train) & set(val)) # inersection

In [24]:
# val_info = train_info[train_info.NAME.isin(val)].sample(frac=1).reset_index(drop=True) # shuffle

In [25]:
# train_info = train_info[train_info.NAME.isin(train)].sample(frac=1).reset_index(drop=True) # shuffle

In [26]:
# val_info

In [27]:
# train_info

In [28]:
from torch.utils.data import Dataset, DataLoader
import PIL
import torchvision.transforms.functional as fn
!pip install torch_snippets
from torch_snippets import *

class FoodDataset(Dataset):
  def __init__(self, x, y, img_size=256, aug=None):
    self.y = y
    self.x = x 
    self.aug = aug
    self.img_size=img_size # CHANGE THIS for each model
    # Normalize does the following for each channel:
      # image = (image - mean) / std
    self.normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])

  def __preprocess_image(self, im):

    # im = cv2.resize(im, (self.img_size, self.img_size))
    im = resize(im, self.img_size)
    # Specify that augmentation must be done if the augmentation object is
    # provided. This is useful is we need to perform augmentation on
    # training data but not on validation data
    if self.aug: im=self.aug.augment_image(im)

    #  While leveraging pre-trained models, it is mandatory to resize,
    # permute, and then normalize images (as appropriate for that pretrained model), where the images are first scaled to a value between
    # 0 and 1 across the 3 channels and then normalized to a mean of
    # [0.485, 0.456, 0.406] and a standard deviation of [0.229, 0.224, 0.225]
    # across the RGB channels.
    im = torch.tensor(im).permute(2,0,1)
    im = self.normalize(im/255.)
    return im[None]
    
  def __getitem__(self, ix):
    f = self.x[ix]
    target = self.y[ix]

    # im = cv2.imread(f) 
    # im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    im = read(f, 1)
    #return im.float().to(device), torch.tensor([target]).float().to(device)
    return im, target

  def __len__(self): return len(self.x)

  # In general, we leverage the collate_fn method when we have to
  # perform heavy computations. This is because performing such
  # computations on a batch of images in one go is faster than doing it
  # one image at a time.

  # Define collate_fn, which takes the batch of data as input:
  def collate_fn(self, batch):
    """
       batch: is a list of tuples with (example, label)
             where 'example' is a tensor of arbitrary shape
             and label is scalar
    """

    ims, classes = [], []
    for im, target in batch:
      im = self.__preprocess_image(im)
      ims.append(im)
      # target = np.array(target).astype("float64")
      classes.append(float(target))

    classes = [torch.tensor(x).to(device).float() for x in classes]
    classes = torch.stack(classes).type(torch.LongTensor).to(device)
    ims = torch.cat(ims).to(device)

    # -------------------------------------------------
    # Separate the batch of images and their classes into two different variables
    # ims, classes = list(zip(*batch))
    
    # Specify that augmentation must be done if the augmentation object is
    # provided. This is useful is we need to perform augmentation on
    # training data but not on validation data
    # if self.aug: ims=self.aug.augment_images(images=[t.cpu().numpy() for t in ims])

    # Create tensors of images, along with scaling data, by dividing the image shape by 255    
    # ims = np.array([t.cpu().numpy() for t in ims])
    # classes = np.array([t.cpu().numpy() for t in classes])
    
    # ims = torch.tensor(ims)[:,:,:,:].to(device)#/255.
    # classes = torch.tensor(classes).type(torch.LongTensor).to(device)
    # -------------------------------------------------
    
    return ims, classes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [29]:
from imgaug import augmenters as iaa
import random

def get_random_scale():
  return random.uniform(1.0, 1.5)

def get_random_translation():
  return {'x':random.randint(-20,20),'y':random.randint(-20,20)}

def get_random_flip():
  r = random.randint(0,100)
  if r%2==0:
    return 0.5
  else:
    return 0



aug = iaa.Sequential([
                  iaa.Affine(rotate=(0,360), translate_px=get_random_translation(), scale=get_random_scale(), fit_output=False, mode="constant"),
                  iaa.SaltAndPepper(random.uniform(0, 0.2)),
                  iaa.GaussianBlur(sigma=(0.0,2.0)),
                  iaa.Flipud(get_random_flip()),
                  iaa.Fliplr(get_random_flip()),
                  # iaa.LinearContrast(0.5), # https://imgaug.readthedocs.io/en/latest/source/overview/contrast.html?highlight=LinearContrast#linearcontrast
                  iaa.Multiply((0.5, 1.5)), # https://imgaug.readthedocs.io/en/latest/source/overview/arithmetic.html?highlight=Multiply#multiply
                  ])




In [30]:
def get_data(batch_val=128, batch_train=512, batch_test=64, img_size=256):
  # types = [f"Type_{x}" for x in range(NUM_CLASSES)]
  tr_images = train_info.PATH.to_numpy()
  tr_targets = train_info.TYPE.to_numpy()

  # val_images = val_info.PATH.to_numpy()
  # val_targets = val_info.TYPE.to_numpy()
  
  test_images = test_info.PATH.to_numpy()
  test_targets = test_info.TYPE.to_numpy()
  
  test_images_degraded = test_info_degraded.PATH.to_numpy()
  test_targets_degraded = test_info_degraded.TYPE.to_numpy()


  train = FoodDataset(tr_images, tr_targets, img_size, aug=aug)
  # val = FoodDataset(val_images, val_targets, img_size)
  test = FoodDataset(test_images, test_targets, img_size)
  test_degraded = FoodDataset(test_images_degraded, test_targets_degraded, img_size)

  # Next, we define the DataLoader, along with the object's collate_fn method, as follows:

  trn_dl = DataLoader(train, batch_size=batch_train, collate_fn=train.collate_fn,shuffle=True, drop_last=True)
  # val_dl = DataLoader(val, batch_size=batch_val, collate_fn=val.collate_fn)
  test_dl = DataLoader(test, batch_size=batch_test, collate_fn=test.collate_fn)
  test_dl_degraded = DataLoader(test_degraded, batch_size=batch_test, collate_fn=test.collate_fn)


  return trn_dl, test_dl, test_dl_degraded

In [31]:
trn_dl, test_dl, test_dl_degraded = get_data()


# Model

In [32]:
import torchvision
import torch.nn as nn
import torch
import torch.nn.functional as F
from torchvision import transforms,models,datasets
!pip install torch_summary
from torchsummary import summary
device = 'cuda' if torch.cuda.is_available() else 'cpu'

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms,models,datasets
import matplotlib.pyplot as plt
from PIL import Image
from torch import optim
device = 'cuda' if torch.cuda.is_available() else 'cpu'
import cv2, glob, numpy as np, pandas as pd
from glob import glob
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [33]:
def get_model():
  model = models.swin_v2_t(weights='DEFAULT').to(device)

  print(model)

  # Specify that we want to freeze all the parameters in the model
  # downloaded previously
  # for child in model.children():
  #   print(child)
  
  for param in model.parameters():
    param.requires_grad = False

  
  for param in model.features[7].parameters():
    param.requires_grad = True
  
  for param in model.features[6].parameters():
    param.requires_grad = True
  
  
  # Replace the avgpool module to return a feature map of size 1 x 1
  # instead of 7 x 7, in other words, the output is now going to be
  # batch_size x 512 (example) x 1 x 1:

  model.avgpool = nn.AdaptiveAvgPool2d(output_size=(1,1))

  # The layer above, nn.AdaptiveAvgPool2d, is yet
  # another pooling layer with a twist. We specify the output feature
  # map size instead. The layer automatically computes the kernel size
  # so that the specified feature map size is returned. For example, if the
  # input feature map size dimensions were batch_size x 512 x k
  # x k, then the pooling kernel size is going to be k x k. The major
  # advantage with this layer is that whatever the input size, the output
  # from this layer is always fixed and, hence, the neural network can
  # accept images of any height and width.




  # Define the classifier module of the model, where we first flatten
  # the output of the avgpool module, connect the 512 units to the 256
  # units, and perform an activation prior to connecting to the output
  # layer:

  model.head = nn.Sequential(
                                    nn.Linear(768, 512),
                                    nn.ReLU(),
                                    nn.Dropout(0.2),
                                    nn.Linear(512, NUM_CLASSES)
                                    )
  
  
  loss_fn = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(),lr= 1e-4)
  return model.to(device), loss_fn, optimizer

def get_saved_model(model):
  convnext_path = PROJ_PATH+"SwinTransformer.pth"
  state_dict = torch.load(convnext_path)
  model.load_state_dict(state_dict['state'])
  return model.to(device), state_dict['epoch']
  # model(torch.tensor(val).float().to(device))
  

In [34]:
model, loss_fn, optimizer = get_model()
model, n_epochs = get_saved_model(model)


Downloading: "https://download.pytorch.org/models/swin_v2_t-b137f0e2.pth" to /root/.cache/torch/hub/checkpoints/swin_v2_t-b137f0e2.pth
100%|██████████| 109M/109M [00:01<00:00, 66.9MB/s]


SwinTransformer(
  (features): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
      (1): Permute()
      (2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
    )
    (1): Sequential(
      (0): SwinTransformerBlockV2(
        (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
        (attn): ShiftedWindowAttentionV2(
          (qkv): Linear(in_features=96, out_features=288, bias=True)
          (proj): Linear(in_features=96, out_features=96, bias=True)
          (cpb_mlp): Sequential(
            (0): Linear(in_features=2, out_features=512, bias=True)
            (1): ReLU(inplace=True)
            (2): Linear(in_features=512, out_features=3, bias=False)
          )
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
        (norm2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (0): Linear(in_features=96, out_features=384, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=384, out_features=96, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (1): SwinTransformerBlockV2(
        (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
        (attn): ShiftedWindowAttentionV2(
          (qkv): Linear(in_features=96, out_features=288, bias=True)
          (proj): Linear(in_features=96, out_features=96, bias=True)
          (cpb_mlp): Sequential(
            (0): Linear(in_features=2, out_features=512, bias=True)
            (1): ReLU(inplace=True)
            (2): Linear(in_features=512, out_features=3, bias=False)
          )
        )
        (stochastic_depth): StochasticDepth(p=0.018181818181818184, mode=row)
        (norm2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (0): Linear(in_features=96, out_features=384, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=384, out_features=96, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (2): PatchMergingV2(
      (reduction): Linear(in_features=384, out_features=192, bias=False)
      (norm): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
    )
    (3): Sequential(
      (0): SwinTransformerBlockV2(
        (norm1): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
        (attn): ShiftedWindowAttentionV2(
          (qkv): Linear(in_features=192, out_features=576, bias=True)
          (proj): Linear(in_features=192, out_features=192, bias=True)
          (cpb_mlp): Sequential(
            (0): Linear(in_features=2, out_features=512, bias=True)
            (1): ReLU(inplace=True)
            (2): Linear(in_features=512, out_features=6, bias=False)
          )
        )
        (stochastic_depth): StochasticDepth(p=0.03636363636363637, mode=row)
        (norm2): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (0): Linear(in_features=192, out_features=768, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=768, out_features=192, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (1): SwinTransformerBlockV2(
        (norm1): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
        (attn): ShiftedWindowAttentionV2(
          (qkv): Linear(in_features=192, out_features=576, bias=True)
          (proj): Linear(in_features=192, out_features=192, bias=True)
          (cpb_mlp): Sequential(
            (0): Linear(in_features=2, out_features=512, bias=True)
            (1): ReLU(inplace=True)
            (2): Linear(in_features=512, out_features=6, bias=False)
          )
        )
        (stochastic_depth): StochasticDepth(p=0.05454545454545456, mode=row)
        (norm2): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
  

In [35]:
newmodel = torch.nn.Sequential(*(list(model.children())[:-1]))
print(newmodel)

Sequential(
  (0): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
      (1): Permute()
      (2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
    )
    (1): Sequential(
      (0): SwinTransformerBlockV2(
        (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
        (attn): ShiftedWindowAttentionV2(
          (qkv): Linear(in_features=96, out_features=288, bias=True)
          (proj): Linear(in_features=96, out_features=96, bias=True)
          (cpb_mlp): Sequential(
            (0): Linear(in_features=2, out_features=512, bias=True)
            (1): ReLU(inplace=True)
            (2): Linear(in_features=512, out_features=3, bias=False)
          )
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
        (norm2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (0): Linear(in_features=96, out_features=384, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=384, out_features=96, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (1): SwinTransformerBlockV2(
        (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
        (attn): ShiftedWindowAttentionV2(
          (qkv): Linear(in_features=96, out_features=288, bias=True)
          (proj): Linear(in_features=96, out_features=96, bias=True)
          (cpb_mlp): Sequential(
            (0): Linear(in_features=2, out_features=512, bias=True)
            (1): ReLU(inplace=True)
            (2): Linear(in_features=512, out_features=3, bias=False)
          )
        )
        (stochastic_depth): StochasticDepth(p=0.018181818181818184, mode=row)
        (norm2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (0): Linear(in_features=96, out_features=384, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=384, out_features=96, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (2): PatchMergingV2(
      (reduction): Linear(in_features=384, out_features=192, bias=False)
      (norm): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
    )
    (3): Sequential(
      (0): SwinTransformerBlockV2(
        (norm1): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
        (attn): ShiftedWindowAttentionV2(
          (qkv): Linear(in_features=192, out_features=576, bias=True)
          (proj): Linear(in_features=192, out_features=192, bias=True)
          (cpb_mlp): Sequential(
            (0): Linear(in_features=2, out_features=512, bias=True)
            (1): ReLU(inplace=True)
            (2): Linear(in_features=512, out_features=6, bias=False)
          )
        )
        (stochastic_depth): StochasticDepth(p=0.03636363636363637, mode=row)
        (norm2): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (0): Linear(in_features=192, out_features=768, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=768, out_features=192, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (1): SwinTransformerBlockV2(
        (norm1): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
        (attn): ShiftedWindowAttentionV2(
          (qkv): Linear(in_features=192, out_features=576, bias=True)
          (proj): Linear(in_features=192, out_features=192, bias=True)
          (cpb_mlp): Sequential(
            (0): Linear(in_features=2, out_features=512, bias=True)
            (1): ReLU(inplace=True)
            (2): Linear(in_features=512, out_features=6, bias=False)
          )
        )
        (stochastic_depth): StochasticDepth(p=0.05454545454545456, mode=row)
        (norm2): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (0):

In [36]:
model.__class__.__name__

'SwinTransformer'

In [37]:
newmodel.__class__.__name__

'Sequential'

In [38]:
@torch.no_grad()
def get_features(x, y, newmodel):
  labels, preds = [], []
  newmodel.eval() # <- let's wait till we get to dropout section
  prediction = newmodel(x)
  labels.append(y.cpu().detach().numpy())
  preds.append(prediction.cpu().detach().numpy())
  return np.array(preds), np.array(labels)

In [39]:
TEST='test'
TEST_DEGRADED = 'test_degraded'
def get_all_labels_and_preds_features(dataset):
  all_labels, all_preds = np.array([]), np.array([])
  stop = False
  for ix, batch in enumerate(iter(dataset)):
    x, y = batch
    preds, labels = get_features(x, y, newmodel)
    if stop==False:
      stop=True
      all_labels = labels
      all_preds = preds
    else:
      all_labels = np.append(all_labels, labels, axis=1)
      all_preds = np.append(all_preds, preds, axis=1)

  return np.squeeze(all_labels), np.squeeze(all_preds)


def save_data(all_labels, all_preds, type_data=''):
  with open(PROJ_PATH+'all_preds_squeeze'+type_data+'.npy', 'wb') as f:
    np.save(f, all_preds)

  with open(PROJ_PATH+'all_labels_squeeze'+type_data+'.npy', 'wb') as f:
    np.save(f, all_labels)


def load_data(type_data=''):
  with open(PROJ_PATH+'all_preds_squeeze'+type_data+'.npy', 'rb') as input:
    all_preds = np.load(input)

  
  with open(PROJ_PATH+'all_labels_squeeze'+type_data+'.npy', 'rb') as input:
      all_labels = np.load(input)

  print(all_labels.shape, all_preds.shape)
  return all_labels, all_preds



In [40]:
# all_labels_test_degraded, all_preds_test_degraded = get_all_labels_and_preds_features(test_dl_degraded)

In [41]:
# save_data(all_labels_test_degraded, all_preds_test_degraded, TEST_DEGRADED)

In [42]:
all_labels_train, all_preds_train  = load_data()

(128000,)
(128000, 768)

In [43]:
all_labels_test_degraded, all_preds_test_degraded  = load_data(TEST_DEGRADED)

(11994,)
(11994, 768)

In [44]:
all_labels_test, all_preds_test = load_data(TEST)

(11994,)
(11994, 768)

In [45]:
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(n_neighbors=5, 
                       algorithm='brute',
                       metric='minkowski',
                       p=2)
knn.fit(all_preds_train)

NearestNeighbors(algorithm='brute')

## Similarity Retrieval - Test Set

In [141]:
import random 

# def get_random_img_from_py_dataset(data):
#   for ix, batch in enumerate(iter(data)):
    

In [142]:
closest_distances, indices = knn.kneighbors(all_preds_test, n_neighbors=3) # find istances closest in training set

In [143]:
closest_distances.shape, indices.shape

((11994, 3), (11994, 3))

In [144]:
q_n = 0
q = closest_distances.shape[0]

for idx, l in enumerate(indices):
  gt = all_labels_test_degraded[idx]  
  predicted_lab = all_labels_train[l]
  # print(gt, predicted_lab)
  # break
  for lab in predicted_lab:
    if gt == lab:
      q_n+=1


P_n_deg = q_n / q
print("P(n) test set: ", P_n_deg)


  # random_sample = get_random_sample_from_class(test_info_degraded, lab)
  # path_random_sample = random_sample.PATH.to_numpy()[0]
  # img = get_img_from_path(path_random_sample)
  # type_name = id_to_type[lab]

  # plt.subplot(3,3,i)
  # i+=1 
  # img = get_img_from_path(path_random_sample)
  # plt.imshow(img)
  # plt.title(type_name)
  # plt.axis("off")


P(n) per il test set:  1.2036851759212939

## Similarity Retrieval - Degraded Dataset

In [ ]:
import random 

# def get_random_img_from_py_dataset(data):
#   for ix, batch in enumerate(iter(data)):
    

In [ ]:
closest_distances, indices = knn.kneighbors(all_preds_test_degraded, n_neighbors=3) # find istances closest in training set

In [ ]:
closest_distances.shape, indices.shape

((11994, 3), (11994, 3))

In [ ]:
q_n = 0
q = closest_distances.shape[0]

for idx, l in enumerate(indices):
  gt = all_labels_test_degraded[idx]  
  predicted_lab = all_labels_train[l]
  # print(gt, predicted_lab)
  # break
  for lab in predicted_lab:
    if gt == lab:
      q_n+=1


P_n_deg = q_n / q
print("P(n) test set degraded: ", P_n_deg)


  # random_sample = get_random_sample_from_class(test_info_degraded, lab)
  # path_random_sample = random_sample.PATH.to_numpy()[0]
  # img = get_img_from_path(path_random_sample)
  # type_name = id_to_type[lab]

  # plt.subplot(3,3,i)
  # i+=1 
  # img = get_img_from_path(path_random_sample)
  # plt.imshow(img)
  # plt.title(type_name)
  # plt.axis("off")


P(n) per il test set degraded:  0.5907120226780057